In [1]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import KFold

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import Sampler

In [4]:
class Args:
    def __init__(self):
        self.device = "cuda"  # cuda:number or cpu
        self.data = "gdsc2"  # Dataset{gdsc or ccle}


args = Args()
res, drug_feature, exprs, mut, cna, null_mask, pos_num = load_data(args)

load gdsc2


In [5]:
epochs = []
true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()
k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=42)
device = "cuda"

for i, (train_index, test_index) in enumerate(kfold.split(np.arange(pos_num))):
    sampler = Sampler(res, train_index, test_index, null_mask, i)
    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_feature,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    epochs.append(epoch)
    true_datas = pd.concat([true_datas, translate_result(true_data)])
    predict_datas = pd.concat([predict_datas, translate_result(predict_data)])

epoch:   0 loss:1.607321 auc:0.6114
epoch:  20 loss:0.268162 auc:0.9485
epoch:  40 loss:0.213811 auc:0.9529
epoch:  60 loss:0.171863 auc:0.9538
epoch:  80 loss:0.159635 auc:0.9570
epoch: 100 loss:0.151458 auc:0.9608
epoch: 120 loss:0.142384 auc:0.9656
epoch: 140 loss:0.136613 auc:0.9679
epoch: 160 loss:0.134276 auc:0.9686
epoch: 180 loss:0.132795 auc:0.9692
epoch: 200 loss:0.131383 auc:0.9696
epoch: 220 loss:0.131463 auc:0.9701
epoch: 240 loss:0.129064 auc:0.9704
epoch: 260 loss:0.128086 auc:0.9708
epoch: 280 loss:0.126556 auc:0.9714
epoch: 300 loss:0.125986 auc:0.9717
epoch: 320 loss:0.124130 auc:0.9722
epoch: 340 loss:0.122764 auc:0.9728
epoch: 360 loss:0.121860 auc:0.9734
epoch: 380 loss:0.123660 auc:0.9738
epoch: 400 loss:0.120959 auc:0.9739
epoch: 420 loss:0.118112 auc:0.9746
epoch: 440 loss:0.116868 auc:0.9749
epoch: 460 loss:0.116800 auc:0.9748
epoch: 480 loss:0.116341 auc:0.9752
epoch: 500 loss:0.114166 auc:0.9758
epoch: 520 loss:0.113347 auc:0.9762
epoch: 540 loss:0.112493 auc

In [6]:
true_datas.reset_index(drop=True).to_csv("true_gdsc2.csv")
predict_datas.reset_index(drop=True).to_csv("pred_gdsc2.csv")